## Speeding up BigQuery queries with BI Engine

To speed up small queries in BigQuery, simply turn on BI Engine.
The Client API remains exactly the same.

Accompanies https://medium.com/@lakshmanok/speeding-up-small-queries-in-bigquery-with-bi-engine-4ac8420a2ef0

#### Queries

In [1]:
from google.cloud import bigquery
from timeit import default_timer as timer
from datetime import timedelta

def show_query(query):
    client = bigquery.Client()
    query_job = client.query(query, bigquery.job.QueryJobConfig(use_query_cache=False))
    df = query_job.result().to_dataframe()
    print("Compute: {} slotms Bytes: {:.1f} MB".format(query_job.slot_millis, query_job.total_bytes_processed/(1024*1024)))
    return df

This query finds the average prescription claim by state.

In [2]:
COST_BY_STATE="""
    SELECT 
       nppes_provider_state, SUM(total_drug_cost)/SUM(total_claim_count) AS avg_cost
    FROM `bigquery-public-data.medicare.part_d_prescriber_2014`
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10
"""
show_query(COST_BY_STATE)

Compute: 11103 slotms Bytes: 460.1 MB


,nppes_provider_state,avg_cost
0,DC,129.704640
1,DE,103.058334
2,NJ,101.909741
3,NY,94.530195
4,CT,94.155857
5,MD,93.189677
6,HI,90.566499
7,TX,85.389806
8,CA,82.732257
9,MI,80.927364


This query finds the average tip based on payment type.

In [3]:
TIPS_BY_TYPE="""
SELECT payment_type, AVG(tips) AS avg_tip
FROM bigquery-public-data.chicago_taxi_trips.taxi_trips
GROUP BY 1
ORDER BY 2 DESC
"""
show_query(TIPS_BY_TYPE)

Compute: 60073 slotms Bytes: 3142.0 MB


,payment_type,avg_tip
0,Credit Card,3.387041
1,Split,3.201069
2,Mobile,3.090545
3,Way2ride,2.411479
4,No Charge,0.713678
5,Pcard,0.232626
6,Unknown,0.204928
7,Prcard,0.177203
8,Dispute,0.008203
9,Prepaid,0.008201


This query uses a discrete number for grouping and will be inherently faster than the names queries (which use strings).
This query finds the sites with the worst (on average) air quality

In [4]:
AIR_QUALITY="""
            SELECT
               site_num,
               ANY_VALUE(state_name) AS state,
               AVG(aqi) as air_quality_index,
            FROM `bigquery-public-data.epa_historical_air_quality.pm10_daily_summary`
            GROUP BY site_num
            ORDER BY air_quality_index DESC
            LIMIT 10
"""
show_query(AIR_QUALITY)

Compute: 1272 slotms Bytes: 104.2 MB


,site_num,state,air_quality_index
0,8012,Country Of Mexico,81.027027
1,3013,Arizona,68.102236
2,7030,Arizona,61.595819
3,3015,Arizona,57.420390
4,0241,California,54.416667
5,3008,Pennsylvania,51.125606
6,0149,Pennsylvania,49.254864
7,1999,California,48.589238
8,2306,Guam,46.846154
9,3011,Arizona,45.814396


### Without BI Engine

Time it. Note that I am measuring the time taken on the server
using query_job.started and query_job.ended.
This takes out variability due to the time it takes to send the query
over the network to the BigQuery API.

In [6]:
from google.cloud import bigquery
from timeit import default_timer as timer
from datetime import timedelta

# Construct a BigQuery client object.
client = bigquery.Client()

def run_query(query, n=5):
    tot_slotmillis, tot_timeelapsed = 0, timedelta(0)
    for iter in range(n):
        query_job = client.query(query, bigquery.job.QueryJobConfig(use_query_cache=False))
        df = query_job.result().to_dataframe()
        tot_timeelapsed += (query_job.ended - query_job.started)
        tot_slotmillis += query_job.slot_millis
    print("Job stat: slot_mills={} server_time={}".format(tot_slotmillis/n, tot_timeelapsed/n))

Here, I'm running the query without BI Engine turned on.

In [7]:
run_query(COST_BY_STATE)

Job stat: slot_mills=6425.2 server_time=0:00:00.611400


In [8]:
run_query(TIPS_BY_TYPE)

Job stat: slot_mills=47012.2 server_time=0:00:00.829200


In [9]:
run_query(AIR_QUALITY)

Job stat: slot_mills=1067.8 server_time=0:00:00.510400


The slot milliseconds is a proxy for the cost if you have a reservation -- it measures how much your BigQuery slots are getting used.
The server_time is the time taken to process the request (we don't measure the network roundtrip time because it's going to be the
same whether or not you use BI Engine).

### With BI Engine

Then, I went to the web console and turned on a 10 GB BI Engine reservation (monthly cost: $300).
Note: It seems to take about 3 minutes for the memory to become available, so this is something
you should consider doing for a few hours at least, not on a per-query basis.

In [31]:
run_query(COST_BY_STATE)

Job stat: slot_mills=324.2 server_time=0:00:00.253200


In [32]:
run_query(TIPS_BY_TYPE)

Job stat: slot_mills=4036.8 server_time=0:00:00.377400


In [33]:
run_query(AIR_QUALITY)

Job stat: slot_mills=132.0 server_time=0:00:00.267200


As you can see, I got cost improvements of 8x to 20x and time improvements of 2x to 2.5x. My code did not change.

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License